# 第8章 : 拡散モデルの理論

## VAEから拡散モデルへ
VAEでは潜在変数が1つである。これを複数個にして階層化したモデルを階層化VAEという。潜在変数が$T$個である階層化VAEは、以下の式で表される

$$
\begin{align}
p_{\boldsymbol{\theta}_T}\left(\boldsymbol{z}_{T} \mid \boldsymbol{z}_{T-1}\right),\cdots, p_{\boldsymbol{\theta}_2}\left(\boldsymbol{z}_1 \mid \boldsymbol{z}_2\right),p_{\boldsymbol{\theta}_1}\left(\boldsymbol{x} \mid \boldsymbol{z}_1\right) \\
q_{\boldsymbol{\phi}_T}\left(\boldsymbol{z}_{T-1} \mid \boldsymbol{z}_{T}\right), \cdots, q_{\boldsymbol{\phi}_2}\left(\boldsymbol{z}_2 \mid \boldsymbol{z}_1\right),q_{\boldsymbol{\phi}_1}\left(\boldsymbol{z}_1 \mid \boldsymbol{x}\right) \\

\end{align}
$$

これはマルコフ過程である。

これは全部で$2T$個のニューラルネットワークが必要だが、拡散モデルでは以下の方法によって1つのニューラルネットワークだけで済む
- 観測変数と潜在変数の次元を同じにする
- エンコーダは固定の正規分布によるノイズを追加する
よって、以下のように表される

$$
\begin{align}
p_{\boldsymbol{\theta}}\left(\boldsymbol{z}_T \mid \boldsymbol{z}_{T-1}\right),\cdots, p_{\boldsymbol{\theta}}\left(\boldsymbol{z}_1 \mid \boldsymbol{z}_2\right),p_{\boldsymbol{\theta}}\left(\boldsymbol{x} \mid \boldsymbol{z}_1\right) \\
q\left(\boldsymbol{z}_{T-1} \mid \boldsymbol{z}_{T}\right), \cdots, q\left(\boldsymbol{z}_2 \mid \boldsymbol{z}_1\right),q\left(\boldsymbol{z}_1 \mid \boldsymbol{x}\right) \\

\end{align}
$$

このように全ての潜在変数の次元が等しいため1つのパラメータ$\theta$でかける。

上の式を逆拡散過程、下の式を拡散過程という。

## 拡散過程

$$
\begin{align}
q\left(\boldsymbol{x}_t \mid \boldsymbol{x}_{t-1} \right) = \mathcal{N}\left( \boldsymbol{x}_t; \sqrt{1 - \beta_t} \boldsymbol{x}_{t-1}, \beta_t \boldsymbol{I} \right)
\end{align}
$$

このとき、
$$
p\left(\boldsymbol{x}_T \right) \approx \left(\boldsymbol{x}_T; \boldsymbol{0},\boldsymbol{I}\right)
$$

が成り立つ。

拡散過程の式は変数変換トリックを使うと
$$
\boldsymbol{\epsilon} \sim \mathcal{N}\left(\boldsymbol{\epsilon};\boldsymbol{0},\boldsymbol{I}\right) \\
\boldsymbol{x}_t = \sqrt{1-\beta_t}\boldsymbol{x}_{t-1} + \sqrt{\beta_t}\boldsymbol{\epsilon}
$$
とかける。
これは、前データの$\boldsymbol{x}_{t-1}$を少しだけスケールダウンして小さなノイズを加えるということである。スケールダウンするのはノイズ加算後の分散が大きくなることを防ぐためである。

## 逆拡散過程
潜在変数の次元が全て等しいことから1つのニューラルネットワークだけが必要になる。入力はノイズ付与された画像$\boldsymbol{x}_t$と時刻$t$である。
$$
\begin{align}
\hat{\boldsymbol{x}}_{t-1} = \mathrm{NeuralNet}\left(\boldsymbol{x}_t,t;\boldsymbol{\theta}\right) \\
p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t\right) = \mathcal{N}\left(\boldsymbol{x}_{t-1};\hat{\boldsymbol{x}}_{t-1},\boldsymbol{I}\right)
\end{align}
$$

ここでは単純化のためにNNによる出力を平均だけにして、共分散行列を$\boldsymbol{I}$としている。

## ELBO
拡散モデルの学習はVAEと同じく対数尤度の下界であるELBOを最大化することを考える。
拡散モデルのELBOは、VAEと同じく導出できる。VAEでは、
$$
\begin{align}
\mathrm{ELBO}\left(\boldsymbol{x};\boldsymbol{\theta},\boldsymbol{\phi}\right) &= \int q_{\phi}\left(\boldsymbol{z} \mid \boldsymbol{x}\right)\log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x},\boldsymbol{z}\right)}{q_{\boldsymbol{\phi}}\left(\boldsymbol{z} \mid \boldsymbol{x} \right)} d\boldsymbol{z} \\
&= \mathbb{E}_{q_{\boldsymbol{\phi}}\left(\boldsymbol{z}\mid\boldsymbol{x}\right)} \left[ \log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x},\boldsymbol{z}\right)}{q_{\boldsymbol{\phi}\left(\boldsymbol{z} \mid \boldsymbol{x} \right)}} \right]
\end{align}
$$
であった。そのため、拡散モデルでは
$$
\begin{align}
\mathrm{ELBO}\left(\boldsymbol{x};\boldsymbol{\theta}\right) = \mathbb{E}_{q\left(\boldsymbol{x}_1,\boldsymbol{x}_2,\cdots,\boldsymbol{x}_T \mid \boldsymbol{x}_0\right)} \left[ \log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_0, \boldsymbol{x}_1, \boldsymbol{x}_2, \cdots, \boldsymbol{x}_T\right)}{q \left( \boldsymbol{x}_1, \boldsymbol{x}_2, \cdots, \boldsymbol{x}_T \mid \boldsymbol{x}_0\right)} \right]
\end{align}
$$

とかける。ここで、簡略化のために
$$
\boldsymbol{x}_{0:T} = \boldsymbol{x}_0, \boldsymbol{x}_1, \cdots, \boldsymbol{x}_T
$$
とかくことにする。
よって、

$$
\mathrm{ELBO}\left(\boldsymbol{x};\boldsymbol{\theta}\right) = \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[ \log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{0:T}\right)}{q \left( \boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \right]
$$
とかける。

### ELBOの計算①(サンプルサイズT)
$p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{0:T}\right)$は乗法定理とマルコフ性によって次の式でかける

$$
\begin{align}
p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{0:T}\right) &= p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_0 \mid \boldsymbol{x}_1\right)  p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_1 \mid \boldsymbol{x}_2\right) \dots  p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{T-1} \mid \boldsymbol{x}_T\right) p \left(\boldsymbol{x}_T\right) \\
&= p\left(\boldsymbol{x}_T \right) \prod_{t=1}^{T}{p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}
\end{align}
$$
ここで、$p\left(\boldsymbol{x}_T\right)$ は $\mathcal{N}\left(\boldsymbol{x}_T ; \boldsymbol{0}, \boldsymbol{I}\right)$ である。

拡散過程についても同様に

$$
q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)=\prod_{t=1}^{T}{q \left({\boldsymbol{x}_{t}} \mid \boldsymbol{x}_{t-1}\right)}
$$

とかける。

これより、
$$
\begin{align}
\mathrm{ELBO}\left(\boldsymbol{x}_0;\boldsymbol{\theta}\right) &= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)}\left[ \log \frac{p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{0:T}\right)}{q \left( \boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \right] \\
&= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[ \log \frac{p\left(\boldsymbol{x}_T \right) \prod_{t=1}^{T}{p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}}{\prod_{t=1}^{T}{q \left({\boldsymbol{x}_{t}} \mid \boldsymbol{x}_{t-1}\right)}}\right]\\
&= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[\log \prod_{t=1}^{T}{p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)} + \underbrace{\log \frac{ p\left(\boldsymbol{x}_T \right)}{\prod_{t=1}^{T}{q \left({\boldsymbol{x}_{t}} \mid \boldsymbol{x}_{t-1}\right)}}}_{\boldsymbol{\theta}\text{を含まない項}}\right]
\end{align}
$$

$\boldsymbol{\theta}$に関してELBOを最適化するため第2項は無視できて、

$$
\begin{align}
J \left(\boldsymbol{\theta} \right) &:= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[\log \prod_{t=1}^{T}{p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}\right] \\
&= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[ \sum_{t=1}^{T}{ \log p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}\right]
\end{align}
$$

サンプルサイズを1としてモンテカルロ法で近似すると、
$$
\begin{align}
\boldsymbol{x}_{1:T} & \sim q \left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right) \\
J \left(\boldsymbol{\theta} \right) & \approx \sum_{t=1}^T \log p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t\right)
\end{align}
$$
となる。ここで、$p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t\right)$は次の式で表される
$$
\begin{align}
\hat{\boldsymbol{x}}_{t-1} &= \mathrm{NeuralNet}\left(\boldsymbol{x}_t,t;\boldsymbol{\theta}\right) \\
p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t \right) &= \mathcal{N}\left(\boldsymbol{x}_{t-1}; \hat{\boldsymbol{x}}_{t-1},\boldsymbol{I}\right)
\end{align}
$$

これより、目的関数$J\left(\boldsymbol{\theta}\right)$は、

$$
\begin{align}
J\left(\boldsymbol{\theta}\right) & \approx \sum_{t=1}^{T} \log p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1}\mid\boldsymbol{x}_t\right) \\
&= \sum_{t=1}^{T}\log\mathcal{N}\left(\boldsymbol{x}_{t-1};\hat{\boldsymbol{x}}_{t-1},\boldsymbol{I}\right)\\
&=\sum_{t=0}^{T-1} \log \mathcal{N}\left(\boldsymbol{x}_t;\hat{\boldsymbol{x}}_t,\boldsymbol{I}\right)\\
&=\sum_{t=0}^{T-1} \log \frac{1}{\sqrt{\left(2\pi\right)^D|\boldsymbol{I}|}} \exp \left(-\frac{1}{2}\left(\boldsymbol{x}_t - \hat{\boldsymbol{x}}_t\right)^{\top} \boldsymbol{I}^{-1} \left(\boldsymbol{x}_t - \hat{\boldsymbol{x}}_{t}\right)\right)\\
&=\sum_{t=0}^{T-1} \left(-\frac{1}{2}\left(\boldsymbol{x}_t - \hat{\boldsymbol{x}}_t\right)^{\top}\left(\boldsymbol{x}_t - \hat{\boldsymbol{x}}_t\right) + \log \frac{1}{\sqrt{\left(2\pi\right)^D}}\right) \quad \left(\because|\boldsymbol{I}|=1, \boldsymbol{I}^{-1}=\boldsymbol{I}\right)\\
&=-\frac{1}{2}\sum_{t=0}^{T-1} \left(\boldsymbol{x}_t - \hat{\boldsymbol{x}}_t\right)^{\top}\left(\boldsymbol{x}_t - \hat{\boldsymbol{x}}_t\right) + T \log \frac{1}{\sqrt{\left(2\pi\right)^D}}
\end{align}
$$

定数項を無視して、
$$
\begin{align}
J\left(\boldsymbol{\theta}\right) &\approx -\frac{1}{2}\sum_{t=0}^{T-1} \left(\boldsymbol{x}_t - \hat{\boldsymbol{x}}_t\right)^{\top}\left(\boldsymbol{x}_t - \hat{\boldsymbol{x}}_t\right)\\
&= -\frac{1}{2}\sum_{t=0}^{T-1} \|\boldsymbol{x}_t - \hat{\boldsymbol{x}}_t\|^2_2
\end{align}
$$

### ELBOの計算②(サンプルサイズ2)
上では、ELBOを$T$個のサンプリングデータによって近似した。続いて、2個のサンプリングデータによってELBOを近似する方法を導出する。

まず、$q\left(\boldsymbol{x}_t | \boldsymbol{x}_0\right)$が解析的に表せることを示す。

以下の正規分布の和の再生性を使う
$$
\begin{align}
x &\sim \mathcal{N}\left(\mu_x, \sigma_x^2\right) \\
y &\sim \mathcal{N}\left(\mu_y, \sigma_y^2\right) \\ 
z &:= x + y \\ 
z & \sim \mathcal{N}\left(\mu_x + \mu_y, \sigma_x^2 + \sigma_y^2\right)
\end{align}
$$

拡散過程の式は、
$$
\begin{align}
q\left(\boldsymbol{x}_t \mid \boldsymbol{x}_{t-1} \right) = \mathcal{N}\left( \boldsymbol{x}_t; \sqrt{1 - \beta_t} \boldsymbol{x}_{t-1}, \beta_t \boldsymbol{I} \right)
\end{align}
$$

とかけるが、$\alpha_t = 1 - \beta_t$として

$$
\begin{align}
q\left(\boldsymbol{x}_t \mid \boldsymbol{x}_{t-1} \right) = \mathcal{N}\left( \boldsymbol{x}_t; \sqrt{ \alpha_t} \boldsymbol{x}_{t-1}, \left(1-\alpha_t\right) \boldsymbol{I} \right)
\end{align}
$$
変数変換トリックを使って
$$
\begin{align}
\boldsymbol{\epsilon}_t &\sim \mathcal{N}\left(\boldsymbol{0}, \boldsymbol{I}\right) \\
\boldsymbol{x}_t &= \sqrt{\alpha_t}\boldsymbol{x}_{t-1} + \sqrt{1-\alpha_t}\boldsymbol{\epsilon}_t
\end{align}
$$
$t-1 \to t-2$を考えて

$$
\begin{align}
\boldsymbol{x}_t &= \sqrt{\alpha_t}\boldsymbol{x}_{t-1} + \sqrt{1-\alpha_t}\boldsymbol{\epsilon}_t \\
&= \sqrt{\alpha_t}\left(\sqrt{\alpha_{t-1}}\boldsymbol{x}_{t-2} + \sqrt{1-\alpha_{t-1}}\boldsymbol{\epsilon_{t-1}}\right) + \sqrt{1-\alpha_t}\boldsymbol{\epsilon}_t \\
&= \sqrt{\alpha_t \alpha_{t-1}} \boldsymbol{x}_{t-2} + \sqrt{\alpha_t - \alpha_t\alpha_{t-1}}\boldsymbol{\epsilon}_{t-1} + \sqrt{1-\alpha_t}\boldsymbol{\epsilon}_t \\
&= \sqrt{\alpha_t \alpha_{t-1}}\boldsymbol{x}_{t-2} + \sqrt{1-\alpha_t \alpha_{t-1}}\boldsymbol{\epsilon} \quad \left(\because 正規分布の再生性\right)
\end{align}
$$

これを繰り返すと

$$
\begin{align}
\boldsymbol{x}_t &= \sqrt{\bar{\alpha}_t}\boldsymbol{x}_0 + \sqrt{1-\bar{\alpha}_t}\boldsymbol{\epsilon}\\
\bar{\alpha}_t &= \alpha_t \alpha_{t-1} \dots \alpha_1
\end{align}
$$

よって、
$$
q\left(\boldsymbol{x}_t \mid \boldsymbol{x}_0\right) = \mathcal{N}\left(\boldsymbol{x}_t;\sqrt{\bar{\alpha}_t}\boldsymbol{x}_0,\left(1-\bar{\alpha}_t\right)\boldsymbol{I}\right)
$$
となる。

これを用いてELBOの近似を行う。①と同様にELBOの式を展開して以下を考える。
$$
\begin{align}
J \left(\boldsymbol{\theta} \right) =\mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[ \sum_{t=1}^{T}{ \log p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)}\right]
\end{align}
$$

①ではこれにたいしてサンプルデータ$T$個を用いてモンテカルロ法の近似を使った。ここでは、以下のように展開する

$$
\begin{align}
J \left(\boldsymbol{\theta} \right) &= \mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[ \sum_{t=1}^T \log p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)\right]\\
&= \sum_{t=1}^{T}\mathbb{E}_{q\left(\boldsymbol{x}_{1:T} \mid \boldsymbol{x}_0\right)} \left[  \log p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)\right] \quad \left(\because\text{期待値の線型性} : \mathbb{E}_{p\left(x,y\right)}\left[x+y\right]=\mathbb{E}_{p\left(x\right)}\left[x\right]+\mathbb{E}_{p\left(y\right)}\left[y\right]\right)\\
&= \sum_{t=1}^{T}\mathbb{E}_{q\left(\boldsymbol{x}_{t},\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_0\right)} \left[  \log p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)\right] \quad \left(\because\text{関連する変数の期待値} : \mathbb{E}_{p\left(x,y\right)}\left[f\left(x\right)\right]=\mathbb{E}_{p\left(x\right)}\left[f\left(x\right)\right]\right)
\end{align}
$$

ここで以下の$T$個の和は一様分布の期待値で表せるという性質を使う。
$1$から$T$の値を取る一様分布$u\left(t\right)$について
$$
\begin{align}
\mathbb{E}_{u\left(t\right)}\left[ f\left(t\right)\right] &= \sum_{t=1}^{T}u\left(t\right)f\left(t\right) \\
&= \sum_{t=1}^{T}\frac{1}{T}f\left(t\right) \\
&= \frac{1}{T} \sum_{t=1}^T f\left(t\right)
\end{align}
$$
より、
$$
\sum_{t=1}^T f\left(t\right) = T \mathbb{E}_{u\left(t\right)}\left[f\left(t\right)\right]
$$
のように和を一様分布の期待値で表せる。

これを用いて
$$
\begin{align}
J \left(\boldsymbol{\theta}\right) &= \sum_{t=1}^T \mathbb{E}_{q\left(\boldsymbol{x}_{t-1},\boldsymbol{x}_t \mid \boldsymbol{x}_0\right)}\left[\log p_{\boldsymbol{\theta}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t\right)}\right] \\
&= T \mathbb{E}_{u\left(t\right)} \left[ \mathbb{E}_{q\left(\boldsymbol{x}_{t-1},\boldsymbol{x}_t \mid \boldsymbol{x}_0\right)} \left[ \log p_{\boldsymbol{\theta}} \left({\boldsymbol{x}_{t-1}} \mid \boldsymbol{x}_t\right)\right]\right]
\end{align}
$$
とかける。この二重の期待値を以下のサンプリングで近似する

$$
\begin{align}
t &\sim U\{1,T\} \\
\boldsymbol{x}_{t-1} &\sim q\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_0\right) \\
\boldsymbol{x}_t &\sim q\left(\boldsymbol{x}_t \mid \boldsymbol{x}_{t-1}\right) \\
J\left(\boldsymbol{\theta}\right) &\approx T \log p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t\right)
\end{align}
$$

また、$p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t\right)$は
$$
\begin{align}
\hat{\boldsymbol{x}}_{t-1} &= \mathrm{NeuralNet}\left(\boldsymbol{x}_t,t;\boldsymbol{\theta}\right) \\
p_{\boldsymbol{\theta}}\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_t\right) &= \mathcal{N}\left(\boldsymbol{x}_{t-1};\hat{\boldsymbol{x}}_{t-1}, \boldsymbol{I}\right)
\end{align}
$$
である。よって正規分布の対数尤度に帰着して
$$
J\left(\boldsymbol{\theta}\right) \approx - \frac{T}{2} \| \boldsymbol{x}_{t-1} - \hat{\boldsymbol{x}}_{t-1} \|^2
$$
となる。

$J\left(\boldsymbol{\theta}\right)$の計算をまとめると以下
1. 一様分布$U\{1,T\}から$t$をサンプリング
2. $q\left(\boldsymbol{x}_{t-1} \mid \boldsymbol{x}_0\right)$から$\boldsymbol{x}_{t-1}$をサンプリング
3. $q\left(\boldsymbol{x}_t \mid \boldsymbol{x}_{t-1}\right)$から$\boldsymbol{x}_t$をサンプリング
4. ニューラルネットワークに$\boldsymbol{x}_t$を入力して$\hat{\boldsymbol{x}}_{t-1}$を得る
5. 二乗誤差$\| \boldsymbol{x}_{t-1} - \hat{\boldsymbol{x}}_{t-1} \|^2$を計算

### ELBOの計算③(サンプルサイズ3)